# Community Clustering

In this notebook, we will cluster the graph using various algorithms implemented in cuGraph.  We will then compare the clusters resulting from each algorithm.

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware |
| --------------|------------|------------------|-----------------|----------------|
| Don Acosta    | 07/05/2022 | tested / updated | 22.08 nightly   | DGX Tesla V100 CUDA 11.5

Clustering is the analytic method for finding the highly connected sets of vertices within a graph. It is often used to answer questions like:

* What are the communities within this graph?
* How can the graph be cut into the most cohesive partitions?
* What is the most important group of vertices within this group?

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


<img src="../../img/zachary_graph_clusters.png" width="35%"/>

Because the test data has vertex IDs starting at 1, the auto-renumber feature of cuGraph (mentioned above) will be used so the starting vertex ID is zero for maximum efficiency. The resulting data will then be auto-unrenumbered, making the entire renumbering process transparent to users.

In [1]:
#  Import the cugraph modules
import cugraph
import cudf

In [2]:
# import non cugraph modules
import numpy as np

In [3]:
# Compute clusters
# the clustering calls are very straightforward with the graph being the primary argument
# we are specifying a few optional parameters for this dataset.

def compute_clusters(_graph) :

    # Compute ECG Clusters and normalize the column names
    _e = cugraph.ecg(_graph).rename(columns={'partition': 'cluster'})
    
    # Compute Louvain Clusters 
    _l, modularity = cugraph.louvain(_graph)
    # Normalize the column names
    _l = _l.rename(columns={'partition': 'cluster'})

    # Compute Spectral Balanced Clusters
    _b = cugraph.spectralBalancedCutClustering(_graph, 4, num_eigen_vects=4)

    # Call spectralModularityMaximizationClustering on the graph for 3 clusterstral 
    _m = cugraph.spectralModularityMaximizationClustering(_graph, 4, num_eigen_vects=4)
    return _e, _l, _b, _m

In [4]:
# compare 2 cluster results
def compare_values(algo, v1, v2):
    return (algo.loc[algo['vertex'] == v1]['cluster'].reset_index(drop=True)).equals((algo.loc[algo['vertex'] == v2]['cluster'].reset_index(drop=True)))

This functon builds a matrix to identify which algorithms cluster pairs of vertices together.
The matrix[i][j] element includes a list of the algorithms where i and j are clustered together:
* e = Ensemble Graph Clustering has placed i and j together in a cluster
* l = Louvain community detection has placed i and j together in a cluster
* b = Spectral Balanced Clustering has placed i and j together in a cluster
* m = Spectral Modularity Maximization Clustering has placed i and j together in a cluster.

In [5]:
def create_cluster_matrix(ecg, louvain, spec_balance, spec_mod):
    mat_size = ecg['vertex'].max()
    clust_matrix = np.empty((mat_size+1) * (mat_size+1), dtype='object')
    clust_matrix = clust_matrix.reshape((mat_size+1),(mat_size+1))

    type(ecg['vertex'])

    for id_1 in ecg['vertex'].to_pandas():
        for id_2 in ecg['vertex'].to_pandas():
            clust_matrix[id_1][id_2] = ""
            if (ecg.loc[ecg['vertex'] == id_1]['cluster'].all()) == (ecg.loc[ecg['vertex'] == id_2]['cluster'].all()):
                clust_matrix[id_1][id_2] += "e"
            if (louvain.loc[louvain['vertex'] == id_1]['cluster'].all()) == (louvain.loc[louvain['vertex'] == id_2]['cluster'].all()):
                clust_matrix[id_1][id_2] += "l"
            if (spec_balance.loc[spec_balance['vertex'] == id_1]['cluster'].all()) == (spec_balance.loc[spec_balance['vertex'] == id_2]['cluster'].all()):
                clust_matrix[id_1][id_2] += "b"
            if (spec_mod.loc[spec_mod['vertex'] == id_1]['cluster'].all()) == (spec_mod.loc[spec_mod['vertex'] == id_2]['cluster'].all()):
                clust_matrix[id_1][id_2] += "m"

    return clust_matrix   

In [6]:
def create_cluster_matrix2(ecg, louvain, spec_balance, spec_mod):
    mat_size = ecg['vertex'].max()
    clust_matrix = np.empty((mat_size+1) * (mat_size+1), dtype='object')
    clust_matrix = clust_matrix.reshape((mat_size+1),(mat_size+1))

    type(ecg['vertex'])

    for id_1 in ecg['vertex'].to_pandas():
        for id_2 in ecg['vertex'].to_pandas():
            clust_matrix[id_1][id_2] = ""
            if compare_values(ecg, id_1, id_2):
                clust_matrix[id_1][id_2] += "e"
            if compare_values(louvain, id_1, id_2):
                clust_matrix[id_1][id_2] += "l"
            if compare_values(spec_balance, id_1, id_2):
                clust_matrix[id_1][id_2] += "b"
            if compare_values(spec_mod, id_1, id_2):
                clust_matrix[id_1][id_2] += "m"

    return clust_matrix   

Print the table showing which algorithms group which vertices together

In [7]:
def print_clustering_table(cluster_array):
    import pandas as pd
    from IPython.display import display_html
    df = pd.DataFrame(cluster_array)
    df_styler = df.drop(df.columns[[0]], axis=1).drop(0).style.set_table_attributes("style='display:inline'")
    display_html(df_styler._repr_html_(), raw=True)

## Read the data

In [8]:
# Test file    
datafile='../../data/karate-data.csv'

In [9]:
# read the data using cuDF
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

In [10]:
# The algorithms often also require that there are vertex weights.  Just use 1.0 
gdf["data"] = 1.0

it was that easy to load data

## Create a Graph

In [11]:
# create a Graph - since the data does not start at '0', use the auto-renumbering feature
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr='data', renumber=True)

## Now do all the clustering

In [12]:
_e, _l, _b, _m = compute_clusters(G)

View the clusters for a single algorithm, in this case Ensemble Graph Clustering

In [13]:
_e.to_pandas().groupby('cluster')['vertex'].apply(list)

cluster
0                                     [25, 26, 29, 32]
1                                    [5, 11, 17, 6, 7]
2          [20, 10, 13, 18, 22, 1, 3, 2, 4, 14, 8, 12]
3    [15, 16, 19, 21, 23, 34, 33, 9, 24, 30, 31, 28...
Name: vertex, dtype: object

Generate the cluster comparison matrix to view the results of the clustering algorithms in one structure. Notice, the first row and column are index 0 which is empty since the graph has been numbered/renumbered to start with 1.

In [14]:
clust_comparison = create_cluster_matrix2(_e, _l, _b, _m)

5 5
5 5
5 5
5 5
5 11
5 11
5 11
5 11
5 20
5 20
5 20
5 20
5 25
5 25
5 25
5 25
5 26
5 26
5 26
5 26
5 29
5 29
5 29
5 29
5 10
5 10
5 10
5 10
5 13
5 13
5 13
5 13
5 15
5 15
5 15
5 15
5 16
5 16
5 16
5 16
5 17
5 17
5 17
5 17
5 18
5 18
5 18
5 18
5 19
5 19
5 19
5 19
5 21
5 21
5 21
5 21
5 22
5 22
5 22
5 22
5 23
5 23
5 23
5 23
5 34
5 34
5 34
5 34
5 1
5 1
5 1
5 1
5 33
5 33
5 33
5 33
5 3
5 3
5 3
5 3
5 2
5 2
5 2
5 2
5 4
5 4
5 4
5 4
5 32
5 32
5 32
5 32
5 9
5 9
5 9
5 9
5 14
5 14
5 14
5 14
5 24
5 24
5 24
5 24
5 6
5 6
5 6
5 6
5 7
5 7
5 7
5 7
5 8
5 8
5 8
5 8
5 30
5 30
5 30
5 30
5 31
5 31
5 31
5 31
5 28
5 28
5 28
5 28
5 27
5 27
5 27
5 27
5 12
5 12
5 12
5 12
11 5
11 5
11 5
11 5
11 11
11 11
11 11
11 11
11 20
11 20
11 20
11 20
11 25
11 25
11 25
11 25
11 26
11 26
11 26
11 26
11 29
11 29
11 29
11 29
11 10
11 10
11 10
11 10
11 13
11 13
11 13
11 13
11 15
11 15
11 15
11 15
11 16
11 16
11 16
11 16
11 17
11 17
11 17
11 17
11 18
11 18
11 18
11 18
11 19
11 19
11 19
11 19
11 21
11 21
11 21
11 21
11 22
11 22
11 22
11 22


print the entire algorithm clustering comparison table.

In [ ]:
print_clustering_table(clust_comparison)

An individual point in matrix, in this case (1,11) shows that ECG, Louvain and Spectral balanced clustering put vertices 1 and 11 in the same cluster, but Spectral Modularity Maximization does not.

In [ ]:
print(clust_comparison[1][11])

In [ ]:
_e.to_pandas().groupby('cluster')['vertex'].apply(list)

In [ ]:
# _e.loc[_e['vertex'] == 15]['cluster'].reset_index(drop=True) == _e.loc[_e['vertex'] == 33]['cluster'].reset_index(drop=True)
for id_1 in _e['vertex'].to_pandas():
    for id_2 in _e['vertex'].to_pandas():
        compare_values(_e, id_1, id_2)
